In [7]:
import requests
import pandas as pd

def load_data(symbol, start_date, end_date, interval, output_csv_path):
    api_key = 'de6ee984b4c24a0c9c7a43d7dca8b75e' 
    order = 'asc'
    api_url = f'https://api.twelvedata.com/time_series?symbol={symbol}&start_date={start_date}&end_date={end_date}&interval={interval}&order={order}&apikey={api_key}'
        
    response = requests.get(api_url)
    if response.status_code != 200:
        raise Exception("Error in API request")
        
    data = response.json()
    
    # extract the data from the 'values' key
    data = pd.DataFrame(data['values'])
    data.dropna(inplace=True) # Drop NaN values
        
    # cleaning to numeric
    for col in data.columns[1:]: 
        data[col] = pd.to_numeric(data[col], errors='coerce')
        
    # save to CSV
    data.to_csv(output_csv_path, index=False)
    print(f"Data has been successfully saved to {output_csv_path}")
    
    return data

In [10]:
# first try

symbol = "BTC/USD"
start_date = "2023-01-01"
end_date = "2023-01-31"
interval = "1day"
output_csv_path = "bitcoin_stock_data.csv"

df = load_data(symbol, start_date, end_date, interval, output_csv_path)

Data has been successfully saved to bitcoin_stock_data.csv


In [9]:
symbol = "BTC/USD"
start_date = "2023-02-01"
end_date = "2023-12-31"
interval = "1day"
output_csv_path = "bitcoin_stock_data_2.csv"

df = load_data(symbol, start_date, end_date, interval, output_csv_path)

Data has been successfully saved to bitcoin_stock_data_2.csv


In [11]:
file1 = "bitcoin_stock_data.csv"
file2 = "bitcoin_stock_data_2.csv"

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

combined_df = pd.concat([df1, df2])

# save the combined dataset to a new CSV
combined_output_csv = "bitcoin_daily_2023_stock_data.csv"
combined_df.to_csv(combined_output_csv, index=False)